In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd -q ..

In [10]:
import lcdb
import json
import lcpfn 
import torch as th
from functools import partial
import numpy as np
from matplotlib import pyplot as plt

In [11]:
curve = lcdb.get_curve(3, "sklearn.linear_model.LogisticRegression")

In [ ]:
def readDatasetJson(path):
    f = open(path)

    dataset = json.read(f)
    
    for k in dataset.keys():
        #TURN THEM INTO INT
        pass

    return

In [12]:
anchors, scores_train, scores_valid, scores_test = curve
print(anchors)
print(scores_train)
print(scores_valid)
print(scores_test)

[16, 23, 32, 45, 64, 91, 128, 181, 256, 362, 512, 724, 1024, 1448, 2048, 2588]
[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9375, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9375, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9375, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9375, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9375, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.913, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9565, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9565, 1.0, 1.0, 1.0, 1.0, 0.9565, 1.0, 1.0, 1.0, 1.0, 0.9565, 0.9565, 

In [49]:
def dummyLoader(seq_len_maximum = None, device = None, batch_size = None, seq_len = None, num_features = None, hyperparameters = None, single_eval_pos = None, x = None, y = None, z = None):
    anchors, scores_train, scores_valid, scores_test = curve
    x = th.Tensor(anchors, )
    y = th.mean(th.Tensor(scores_train), dim = 1)
    y_new = th.mean(th.Tensor(scores_test), dim = 1)
    return x, y, y_new

x, y ,z = dummyLoader(curve)
print(x)
print(y)
print(z)

tensor([  16.,   23.,   32.,   45.,   64.,   91.,  128.,  181.,  256.,  362.,
         512.,  724., 1024., 1448., 2048., 2588.])
tensor([0.9975, 0.9951, 0.9920, 0.9876, 0.9829, 0.9782, 0.9768, 0.9719, 0.9682,
        0.9664, 0.9666, 0.9666, 0.9662, 0.9666, 0.9676, 0.9684])
tensor([0.6607, 0.7168, 0.7664, 0.8081, 0.8513, 0.8825, 0.9076, 0.9294, 0.9431,
        0.9517, 0.9574, 0.9606, 0.9628, 0.9651, 0.9673, 0.9686])


In [44]:
def create_get_batch_func(prior):
    return partial(get_batch_domhan, prior=prior)

# function producing batches for PFN training
def get_batch_domhan(
    batch_size,
    seq_len,
    num_features,
    device="cpu",
    noisy_target=True,
    **_,
):
    assert num_features == 1

    x = np.arange(1, seq_len + 1)
    y_target = np.empty((batch_size, seq_len), dtype=float)
    y_noisy = np.empty((batch_size, seq_len), dtype=float)

    for i in range(batch_size):
        curve_func =  [0.9975, 0.9951, 0.9920, 0.9876, 0.9829, 0.9782, 0.9768, 0.9719, 0.9682,
        0.9664, 0.9666, 0.9666, 0.9662, 0.9666, 0.9676, 0.9684]# uses numpy rng
        if noisy_target:
            y_noisy[i] = curve_func[i]
            y_target[i] = y_noisy[i]
        else:
            y_target[i], y_noisy[i] = curve_func[i], curve_func[i]
    # turn numpy arrays into correctly shaped torch tensors & move them to device
    x = (
        th.arange(1, seq_len + 1)
        .repeat((num_features, batch_size, 1))
        .transpose(2, 0)
        .to(device)
    )
    y_target = th.from_numpy(y_target).transpose(1, 0).to(device)
    y_noisy = th.from_numpy(y_noisy).transpose(1, 0).to(device)

    # changes
    x = x.float()
    y_target = y_target.float()
    y_noisy = y_noisy.float()

    return x, y_noisy, y_target

In [45]:
get_batch_func = create_get_batch_func(prior = None)

In [48]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch_domhan, 
                         seq_len=16,
                         emsize=256,
                         nlayers=3,
                         num_borders=4, #THIS HAS SOMETHING TO DO WITH THE NUMBER OF BUCKETS, NEED TO FIGURE OUT TF IS GOING ON WITH
                         lr=0.001,
                         batch_size=1,
                         epochs=3)

nb 4
ys tensor([[0.9975],
        [0.9975],
        [0.9975],
        [0.9975],
        [0.9975],
        [0.9975],
        [0.9975],
        [0.9975],
        [0.9975],
        [0.9975],
        [0.9975],
        [0.9975],
        [0.9975],
        [0.9975],
        [0.9975],
        [0.9975]])
Using 16 y evals to estimate 4 buckets. Cut off the last 0 ys.
Using cpu:0 device
init dist
Not using distributed
DataLoader.__dict__ {'num_steps': 100, 'get_batch_kwargs': {'batch_size': 1, 'eval_pos_seq_len_sampler': <function train.<locals>.eval_pos_seq_len_sampler at 0x000001984E815EE0>, 'seq_len_maximum': 16, 'device': 'cpu:0', 'num_features': 1, 'hyperparameters': {}}, 'num_features': 1}
Style definition: None
Using a Transformer with 1.72 M parameters


ValueError: Expected parameter scale (Tensor of shape ()) of distribution HalfNormal() to satisfy the constraint GreaterThan(lower_bound=0.0), but found invalid values:
0.0